In [ ]:
import xarray as xr
from skimage.morphology import skeletonize
from shapely.geometry import MultiPolygon
import geopandas as gpd
import rioxarray as rxr
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
gen_path = '/Users/emarshall/Desktop/siparcs/xr_book1/'

## Read in vector data

In [ ]:
se_asia = gpd.read_file(gen_path[:-9] + 'data/nsidc0770_15.rgi60.SouthAsiaEast/15_rgi60_SouthAsiaEast.shp')

In [ ]:
se_asia_prj = se_asia.to_crs('+proj=lcc +lat_1=15 +lat_2=65 +lat_0=30 +lon_0=95 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m no_defs')
se_asia_prj

In [ ]:
sample_glacier_prj = se_asia_prj.loc[se_asia_prj['RGIId'] == 'RGI60-15.11909']
sample_glacier_prj

In [ ]:
sample_glacier_series_prj = se_asia_prj.iloc[11908]

Need to convert from geopandas polygon to shapely </br>
*help from here:* https://stackoverflow.com/questions/66926708/converting-a-geopandas-geometry-into-a-shapleys-polygon

In [ ]:
import shapely
shapely.__version__

In [ ]:
attributes = {'id': 11909, 'name':'sample_glacier','valid':True}
centerline = Centerline(sample_glacier_series_prj['geometry'], **attributes)

In [ ]:
type(centerline)
centerline.name

## Read in raster data 
- for once we've exctracted centerline


In [ ]:
itslive = rxr.open_rasterio(gen_path[:-9] + '/data/HMA_G0120_0000.nc').squeeze()

In [ ]:
glacier_raster = itslive.rio.clip(sample_glacier_prj.geometry, sample_glacier_prj.crs)

In [ ]:
just_ice = xr.where(glacier_raster.ice == 1., glacier_raster, np.NaN)

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))

just_ice.v.plot(ax=ax)

## Switching away from centerline (abandoning that env) and trying scikit image - skeletonize



In [ ]:
from skimage.morphology import skeletonize 

In [ ]:
just_ice.ice

In [ ]:
skeleton_ice = skeletonize(just_ice.ice.data)
skeleton_ice_lee = skeletonize(just_ice.ice.data)

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize = (14,4), sharex=True, sharey=True)

ax = axes.ravel()

ax[0].imshow(just_ice.v, cmap=plt.cm.cividis)
ax[0].axis('off')
ax[0].set_title('Glacier velocity', fontsize=14)
ax[0].imshow(skeleton_ice, cmap=plt.cm.Reds, alpha = 0.2)
ax[0].imshow(skeleton_ice_lee, cmap=plt.cm.Greens, alpha = 0.2)


ax[1].imshow(skeleton_ice, cmap=plt.cm.Reds)
ax[1].axis('off')
ax[1].set_title('skeleton', fontsize=14)

ax[2].imshow(skeleton_ice_lee, cmap=plt.cm.Greens)
ax[2].axis('off')
ax[2].set_title('skeleton (lee)', fontsize=14)

fig.tight_layout()
fig.show

In [ ]:
type(skeleton_ice)
skeleton_ice.shape



In [ ]:
just_ice.ice

### end of day 6/8 
- still need to figure out s3 urls
- would be great to figure out adding the skeletonized object as a variable (maybe?) to the itslive xarray object (maybe better to do w/ vectorized indexing than adding as a var)

In [ ]:
skeleton_da = xr.DataArray(
    data = (('y','x'),skeleton_ice),
    dims = ['y','x'],
    coords = ({'x':[just_ice.x.data], 'y':[just_ice.y.data]}),
)

skeleton_da = xr.DataArray(
    data=skeleton_ice,
    dims=["y", "x"],
    coords=dict(
        lon=(["x", "y"], just_ice.x.data),
        lat=(["x", "y"], just_ice.y.data),
        
    ),
    
    )